**1. Libraries**

In [ ]:
from requests_html import HTMLSession
import pandas as pd
import re, os, urllib, shutil
import pandas as pd
import sqlite3

**2. Directories**

In [ ]:
## Creating directories
path_raw = os.path.join(os.getcwd(), 'Projections')
path_national = os.path.join(path_raw, 'National')
path_departmental = os.path.join(path_raw, 'Departmental')
path_municipal = os.path.join(path_raw, 'Municipal')
if os.path.exists(path_raw) == False: 
    os.mkdir(path_raw)
if os.path.exists(path_national) == False: 
    os.mkdir(path_national)
if os.path.exists(path_departmental) == False: 
    os.mkdir(path_departmental)
if os.path.exists(path_municipal) == False: 
    os.mkdir(path_municipal)

**3. Connecting**

In [ ]:
session = HTMLSession()

In [ ]:
url = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/demografia-y-poblacion/proyecciones-de-poblacion'

In [ ]:
response = session.get(url)

**3.1 Scrapping**

In [ ]:
links = response.html.absolute_links

In [ ]:
National = list()
Departmental = list()
Municipal = list()
for link in links:
    if re.search('anexo-(area-sexo-edad-)*proyecciones-poblacion-Nacional[0-9]{4}_[0-9]{4}.xlsx$', link):
        National.append(link)
        continue
    if re.search('anexo-(area-sexo-edad-)*proyecciones-poblacion-departamental_[0-9]{4}-[0-9]{4}.xlsx$', link):
        Departmental.append(link)
        continue
    if re.search('anexo-(area-sexo-edad-)*proyecciones-poblacion-Municipal_[0-9]{4}-[0-9]{4}.xlsx$', link):
        Municipal.append(link)
        continue

**3.2 Download and export**

In [ ]:
def get_csv(links, pattern, path_level,csv_name, level, insert_as):
    
    if level == 'Municipio':
        col_changes = {'DP':'Departamento', 'ÁREA GEOGRÁFICA':'Area', 'DPMP':'Municipio', 'AÑO':'Year'}
        column_names = ['Municipio', 'Sex', 'Area']
        col_rem = ['Sexo_edad','DPNOM','MPIO']
    else:
        col_changes = {'DP':'Departamento', 'ÁREA GEOGRÁFICA':'Area', 'AÑO':'Year'}
        column_names = ['Sex', 'Area']
        col_rem = ['Sexo_edad','DPNOM']
        insert_projection = '''INSERT INTO Projections(departamento_id, area_id, sex_id, year, age, No_people) VALUES'''

    
    for web in links:
        name = re.findall(pattern, web)[0]
        path_doc = os.path.join(path_level, name)
        if os.path.isfile(path_doc) == False:
            print('Reading', path_doc)
            urllib.request.urlretrieve(web, path_doc)
        print('File downloaded')
        df_all = pd.read_excel(path_doc, skiprows=11)
        print(name, 'read')
        cols_to_split = df_all.loc[:,'Hombres_0':].columns.values
        ids = df_all.loc[:,:'ÁREA GEOGRÁFICA'].columns.values
        print('ids and split identified')
        new_all = pd.melt(df_all, id_vars =ids, value_vars =cols_to_split, var_name ='Sexo_edad', value_name ='No_personas')

        print('col to row')
        new_all[["Sex", "Age"]] = new_all.Sexo_edad.str.split(pat='_', n = 1, expand = True)
        print('sex and age splited')
        new_all = new_all.drop(columns=col_rem)
        
        new_all.rename(columns = col_changes, inplace = True)
        print(new_all.head(2))
        # Connecting with the database
        con = sqlite3.connect('Colombia_Demographic_data.sqlite', timeout=30)
        cur = con.cursor()
        
        z = 1
        if insert_as == 'yes' and z == 1:
            ## area's table
            insert_area = 'INSERT INTO Area(name) VALUES'
            insert_area += '('
            
            for area in new_all['Area'].unique():
                try:
                    select_query = 'SELECT id FROM '
                    select_query += 'Area' + ' WHERE name = ? ;'
                    cur.execute(select_query, (area, ))
                    cur.fetchone()[0]
                    continue
                except:
                    insert_area += '"'+str(area) + '"'+'),('
                    
            if insert_area != 'INSERT INTO Area(name) VALUES(':
                insert_area = insert_area[:-2] +';'
                cur.execute(insert_area)
            
            print('area inserted')
            
            ## Sex's table
            insert_sex = 'INSERT INTO Sex(name) VALUES'
            insert_sex += '('
            for sex in new_all['Sex'].unique():
                try:
                    select_query = 'SELECT id FROM '
                    select_query += 'Sex' + ' WHERE name = ? ;'
                    cur.execute(select_query, (sex, ))
                    cur.fetchone()[0]
                    continue
                except:
                    insert_sex += '"'+str(sex) + '"'+'),('
            
            if insert_sex != 'INSERT INTO Sex(name) VALUES(':
                insert_sex = insert_sex[:-2] +';'
                cur.execute(insert_sex)
            con.commit()
            print('area and sex inserted')
            z += 1
            
        if level == 'Municipio':    
            insert_projection = '''INSERT INTO Projections(departamento_id, municipio_id, year, area_id, No_people, sex_id, age) VALUES'''
        else:
            insert_projection = '''INSERT INTO Projections(departamento_id, year , area_id, No_people, sex_id, age) VALUES'''
        print(len(new_all))
        r = 0
        for row in range(0,len(new_all)):
            i = 0
            insert_projection += '('
            print('row:', row)
            for feature in new_all.iloc[row]:
                
                title   = new_all.columns.values[i]
                feature = feature

                if title in column_names:
                    select_query = 'SELECT id FROM '
                    select_query += str(title) + ' WHERE name = ? ;'
                    cur.execute(select_query, (feature, ))
                    try:
                        feature_id = cur.fetchone()[0]
                        insert_projection += '"'+str(feature_id) + '"'+','
                    except:
                        print(title)
                        print(feature)
                        insert_projection += 'NULL,'
                elif pd.isna(feature):
                    insert_projection += 'NULL,'

                else:
                    insert_projection += '"' + str(feature) + '"'+','

                i +=1
            r +=1
            insert_projection = insert_projection[:-1] +'),'
            
            if r >= 10000:
                insert_projection = insert_projection[:-2] +');'
                cur.execute(insert_projection)
                r = 0
                if level == 'Municipio':    
                    insert_projection = '''INSERT INTO Projections(departamento_id, municipio_id, year, area_id, No_people, sex_id, age) VALUES'''
                else:
                    insert_projection = '''INSERT INTO Projections(departamento_id, year , area_id, No_people, sex_id, age) VALUES'''
            
        if r >0:
            insert_projection = insert_projection[:-2] +');'
            print(insert_projection)
            cur.execute(insert_projection)
            print('executed')
            con.commit()
            print('commited')
        con.close() 

In [ ]:
get_csv(National, "Nacional[0-9]{4}_[0-9]{4}.xlsx$", path_national, 'Projections\\National.csv', 'x', 'yes')

In [ ]:
get_csv(Departmental, "departamental_[0-9]{4}-[0-9]{4}.xlsx$", path_departmental, 'Projections\\Departmental.csv', 'd', 'yes')

In [ ]:
get_csv(Municipal, "Municipal_[0-9]{4}-[0-9]{4}.xlsx$", path_municipal,'Projections\\Municipal.csv', 'Municipio', 'no')

In [ ]:
con.close()        

**4. Remove directories**

In [ ]:
shutil.rmtree(path_national)
shutil.rmtree(path_departmental)
shutil.rmtree(path_municipal)